# Importing necesary libraries 

In [1]:
import geemap
import ee
import os
import pandas as pd

In [2]:
#ee.Initialize()


Successfully saved authorization token.


# Download the shapefiles which gives the vector shapes of the Natural Park in Colombia

In [3]:
nat_park = geemap.Map()

In [4]:
pnn = ee.FeatureCollection('projects/spicedfinalproject/assets/colombia_PNN')
los_nevados = pnn.filter(ee.Filter.eq("Nombre", "LOS NEVADOS"))

In [5]:
nat_park.addLayer(pnn, {}, 'PNN_Colombia')
nat_park.centerObject(pnn)
nat_park

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
Los_nev_map = geemap.Map()
Los_nev_map.addLayer(los_nevados, {}, 'Los Nevados')
Los_nev_map.centerObject(los_nevados)
Los_nev_map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# masking clouds
def maskcloud1(image):
  QA60 = image.select(['QA60'])
  return image.updateMask(QA60.lt(1))

# obtaining NDVI (vegetation index)
def get_ndvi(image):
  return image.addBands(image.normalizedDifference(["B8","B4"]).rename('NDVI'))

# calculating NDMI (moisture index)
def get_ndmi(image):
  return image.addBands(image.normalizedDifference(["B8A","B11"]).rename('NDMI'))

#clip the dataset according to the geometry
# needs to be into a function since the clip function is only for ee.image and we have a ee.ImageCollection
def clip_img_nevados(image):
    return image.clip(los_nevados)

# Open data catalog from Sentinel 2

In [ ]:
# Sentinel collection
Sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
    .filterBounds(los_nevados)\
    .filter(ee.Filter.lte('CLOUD_COVERAGE_ASSESSMENT',18))\
    .select('B.*|QA.*')\
    .map(clip_img_nevados)

In [ ]:
#  First image from Sentinel Coll. 
Sentinel2_img1 = (ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
    .filterBounds(los_nevados)\
    .filter(ee.Filter.lte('CLOUD_COVERAGE_ASSESSMENT',18))\
    .map(clip_img_nevados)\
    .select('B.*|QA.*')
    .first())

## Visualization parameter

In [ ]:
# Vegetation Index
Vispara_ndvi = {'bands':'NDVI',
                    'min': -1, 
                    'max': 1,
                    'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 
                        'FCD163', '99B718', '74A901', '66A000', '529400',
                        '3E8601', '207401', '056201', '004C00', '023B01', 
                        '012E01', '011D01', '011301']}
      


In [ ]:
# Moisture Index
Vispara_ndmi = {'bands':'NDMI',
                    'min': -1, 
                    'max': 1, 
                    'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                        'ff0000', 'de0101', 'c21301', 'a71001', '911003']}

In [ ]:
# Precipiation parameters
Vispara_rain = {
  'min': 0.0,
  'max': 30.0,
  'palette': ['1621a2', 'ffffff', '03ffff', '13ff03', 'efff00', 'ffb103', 'ff2300']}

In [ ]:
# Natural Color
nat_color = {'bands': ["B4","B3","B2"],
  'gamma': 1.8330000000000002,
  'max': 7270,
  'min': 286,
  'opacity': 0.99}

In [ ]:
# Infrared
infrared = {'bands': ["B8","B4","B3"],
  'gamma': 1.6,
  'max': 2233.302942508474,
  'min': 364.9824640612618,
  'opacity': 0.99}

In [ ]:
# Short wave
short_wave = {'bands': ["B12","B8A","B4"], #1st sigmoid
  'gamma': 2.098,
  'max': 2135.7597918822803,
  'min': 675.9357833389589,
  'opacity': 1}

In [ ]:
# Vegetation
vegetation = {'bands': ["B11","B8","B2"], #90% stretch
  'gamma': 1.8330000000000002,
  'max': 5948.1,
  'min': 932.9000000000001,
  'opacity': 1}

## add layers to map

In [ ]:
map = geemap.Map()
map.add_basemap('OpenStreetMap')
map

In [ ]:
# add sentinel collection
map.addLayer(Sentinel2, nat_color, 'Natural Color')
# add Natural Parl
map.centerObject(los_nevados)
map.addLayer(los_nevados, {}, 'Los Nevados')

### Checking image properties

In [ ]:
props = geemap.image_props(Sentinel2_img1)
props.getInfo()

In [ ]:
props.get('IMAGE_DATE').getInfo()

In [ ]:
props.get('system:time_start').getInfo()

In [ ]:
props.get('CLOUD_COVERAGE_ASSESSMENT').getInfo()

# Unsupervised Classification 

In [ ]:
# make training data
training = Sentinel2_img1.sample(
    **{
        'region': los_nevados,
        'scale': 100,
        'numPixels': 1500,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

In [ ]:
map.addLayer(training, {}, 'training', False)
map

# ee.Clusterer.wekaKMeans 
Cluster data using the k means algorithm. Can use either the Euclidean distance (default) or the Manhattan distance. If the Manhattan distance is used, then centroids are computed as the component-wise median rather than mean.

Link: https://developers.google.com/earth-engine/apidocs/ee-clusterer-wekakmeans

In [ ]:
# train the clusters with the ee.Cluster.wekaKMeans
# establish number of clusters

n_clusters = 10
wekaKMeans = ee.Clusterer.wekaKMeans(n_clusters).train(training)
wekaKMeans

In [ ]:
# apply model to Single Sentinel image
unsupervised = Sentinel2_img1.cluster(wekaKMeans) #apply the model to Sentinel image 
unsupervised

In [ ]:
# add layer to map
map.addLayer(unsupervised.randomVisualizer(), {}, 'Clusters')